In [1]:
import reproducibility
import datasets
import pandas
import numpy as np
import models
import datasets

from sklearn.ensemble import HistGradientBoostingRegressor
from analysis import evaluate_performance

elements_thresholds = [
    # ("Li_ppm", 200),
    # ("Li_ppm", 1000),
    # ("Sb_ppm", 100),
    # ("Zn_ppm", 500),
    # ("Ag_ppm", 4),
    # ("Au_ppb", 8),
    # ("Sn_ppm", 100),
    ("Cu_ppm", 100),
    ("Pb_ppm", 35),
]

c:\Program Files\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Program Files\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Program Files\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
data_frame = pandas.read_excel("Data/Andes Master Geochem 16 July 2023.xlsx")

X_mass = ["SIO2_wt%", "TIO2_wt%", "AL2O3_wt%", "FE2O3T_wt%", "FEOT_wt%", "MNO_wt%", "MGO_wt%", "CAO_wt%", "NA2O_wt%", "K2O_wt%", "P2O5_wt%", "H2O_wt%", "CR2O3_wt%"]
X_impurities_original=["Li_ppm", "Sc_ppm", "V_ppm", "Cr_ppm", "Co_ppm", "Ni_ppm", "Cu_ppm", "Zn_ppm", "Ga_ppm", "Mo_ppm", "W_ppm", "Sn_ppm", "Sb_ppm", "Rb_ppm", "Sr_ppm", "Y_ppm", "Nb_ppm", "Zr_ppm", "Cs_ppm", "Cd_ppm", "Ba_ppm", "La_ppm", "Ce_ppm", "Pr_ppm", "Nd_ppm", "Sm_ppm", "Eu_ppm", "Gd_ppm", "Tb_ppm", "Dy_ppm", "Ho_ppm", "Er_ppm", "Tm_ppm", "Yb_ppm", "Lu_ppm", "Hf_ppm", "Ta_ppm", "Pb_ppm", "Th_ppm", "U_ppm", "Au_ppb", "Ag_ppm", "S_%", "As_ppm", "Se_ppm", "Te_ppm", "Ge_ppm", "Bi_ppm", "Tl_ppm", "Be_ppm", "B_ppm", "F_ppm", "Cl_ppm", "In_ppm"]
X_ratios=["Rb_K2O", "Rb_Yb", "Rb_La", "K2O_MgO", "Rb_MgO", "U_Ba", "Ba_La", "U_Th", "Rb_Sn", "K2O_Sn", "MnO_MgO"]

data_frame.x_labels = lambda : X_mass + X_impurities_original + X_ratios
data_frame.y_labels = lambda : []

data_frame = datasets.clean(data_frame,  X_mass + X_impurities_original + X_ratios, [])
data_frame = datasets.compute_ratios(data_frame, X_ratios, X_mass + X_impurities_original)
geochemical_analysis = data_frame

def seed_hunt(benchmark_f1: float, benchmark_seed: int, target_element: str, ppm_threshold) -> int:
    for i in range(1, 1000):
        reproducibility.seed_random(i)

        X_impurities = [x for x in X_impurities_original if x != target_element]
        
        x_labels = X_mass + X_impurities + X_ratios
        y_labels = [target_element]

        data_frame.x_labels = lambda : x_labels
        data_frame.y_labels = lambda : y_labels

        geochemical_analysis_measured = geochemical_analysis[x_labels + y_labels].dropna(subset=y_labels)
        geochemical_analysis_train = geochemical_analysis_measured.sample(frac=0.8, random_state=i)
        geochemical_analysis_test = geochemical_analysis_measured.drop(geochemical_analysis_train.index)

        x_train = geochemical_analysis_train[x_labels].to_numpy(np.float32)
        y_train = geochemical_analysis_train[y_labels].to_numpy(np.float32).ravel()
        x_test = geochemical_analysis_test[x_labels].to_numpy(np.float32)
        y_test = geochemical_analysis_test[y_labels].to_numpy(np.float32).ravel()   

        model = HistGradientBoostingRegressor(loss="squared_error", validation_fraction=0.2, min_samples_leaf=5, random_state=i).fit(x_train, y_train)

        y_test_predicted = model.predict(x_test)
        comparison, confusion_matrix = evaluate_performance(y_test, y_test_predicted, ppm_threshold, verbose=0)

        if confusion_matrix.f1_score > benchmark_f1:
            benchmark_seed = i
            benchmark_f1 = confusion_matrix.f1_score
            print(f"Seed for {target_element} {ppm_threshold}ppm: {i} | Accuracy = {int(confusion_matrix.accuracy * 100)} | Precision = {int(confusion_matrix.precision * 100)} | Recall = {int(confusion_matrix.recall * 100)} | F1 = {int(confusion_matrix.f1_score * 100)}")
    
    print(f"Best seed {target_element}:", benchmark_seed, benchmark_f1)

for x in elements_thresholds:
    if x[1]:
        seed_hunt(0, 0, x[0], x[1])

KeyError: 'MnO_MgO'